<a href="https://colab.research.google.com/github/itsmihhhhhh/Luno-CryptoTradingBot/blob/master/ETHGBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install luno_python
!pip install mpl_finance
!pip install pyti
!pip install numpy

In [ ]:
from luno_python.client import Client
c = Client(api_key_id='dfqpu5an2kh7q', api_key_secret='UWjQZBJxoNEOKL1id59VaFBSBORvuAdeR2ZHA6_vdso')
try:
  ticker = c.get_ticker(pair='ETHGBP')
  print(ticker)
except Exception as e:
  print(e)

ticker['last_trade']

In [ ]:
res = c.get_order_book(pair='ETHGBP')
res

In [ ]:
balance = c.get_balances(assets=['ETH','GBP'])
balETH = balance['balance'][0]['balance']
balGBP = balance['balance'][1]['balance']
balGBP
balETH

'0.069352'

In [ ]:
enter_trade = True

In [ ]:
pair = 'ETHGBP'
counter_volume='15'
base_volume='0.0015'

#price = ticker['last_trade']
closePrices = c.list_trades(pair='ETHGBP')
price = closePrices['trades'][0]['price']

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
    global enter_trade

    if BuySell == 'B':
      '''
      if enter_trade == True:
        print("Entered trade already, waiting to sell")
      else:
      '''
      try:
        if (price == float(res['asks'][0]['price'])):
          c.post_market_order(pair='ETHGBP', type='BUY', counter_volume='2')
          print("Buy Successfully at price: ", price)
          enter_trade = True
        else:
          c.post_limit_order(pair='ETHGBP', type='BID', volume='0.07', price = str(float(price)))
          print("Post limit order, Buy Successfully at price: ", price)
          enter_trade = True
      except Exception as e:
          print(e)
          enter_trade = False

    if BuySell == 'S':
      '''
      if enter_trade == False:
        print("Havent entered trade")
      else:
      '''
      try:
        if (price == float(res['bids'][0]['price'])):
          c.post_market_order(pair='ETHGBP', type='SELL', base_volume='0.06')
          print("Sell Successfully at price.", price)
          enter_trade = False
        else:
          c.post_limit_order(pair='ETHGBP', type='ASK', volume='0.07', price = str(float(price)))
          print("   Sell Successfully at price.", price)
          enter_trade = False
      except Exception as e:
          print(e)
          enter_trade = True



In [ ]:
!pip install tulipy
import tulipy as ti

rsiPeriod = 14
upper_rsi = 70.0
lower_rsi = 30.0

In [ ]:
res = c.get_order_book(pair='ETHGBP')
len(res['asks'])

In [ ]:
import sched, time
import numpy as np


#Initiate our scheduler so we can keep checking every minute for new price changes
s = sched.scheduler(time.time, time.sleep)

# This function is run every 5 minutes

def run(sc): 

    #Get prices from Luno
    res = c.get_order_book(pair='ETHGBP')
    closePrices = c.list_trades(pair='ETHGBP')
    pricedata = []
    for x in range(0,len(closePrices['trades'])):
        pricedata.append(float(closePrices['trades'][x]['price']))
    DATA = np.array(pricedata)

    askdata = []
    for x in range(0,len(res['asks'])):
        askdata.append(float(res['asks'][x]['price']))
    DATAA = np.array(askdata)

    biddata = []
    for x in range(0,len(res['bids'])):
        biddata.append(float(res['bids'][x]['price']))
    DATAB = np.array(biddata)

    #Calculate Indicators
    #iRSI = ti.rsi(DATA, period=14)
    BB = ti.bbands(DATA, period=20, stddev=2)
    BB1 = ti.bbands(DATA, period=20, stddev=1)
    '''
    MA = ti.sma(DATA, period=5)
    MAA = ti.sma(DATAA, period=3)
    MAB = ti.sma(DATAB, period=5)
    '''

    '''
    if (MAB[0] < pricedata[0] < MA[0]):
        print("Buying, price is low!")
        enter("B")

    if (MA[0] < pricedata[0] < MAA[0]):
        print("Selling, price is high!")
        enter("S")
    '''

    current_price = float(closePrices['trades'][0]['price'])

    if (BB[1][0] > current_price > BB[0][0]):
        print("Buying, price is low!")
        try:
          c.post_limit_order(pair='ETHGBP', type='BID', volume='0.015', price = ticker['last_trade'])
          print("Post limit order, buy at price: ", ticker['last_trade'])
          enter_trade = True
        except Exception as e:
          print(e)
          enter_trade = False

    if (BB[1][0] < current_price < BB[2][0]):
        print("Selling, price is high!")
        try:
          c.post_limit_order(pair='ETHGBP', type='ASK', volume='0.015', price = pricedata[0])
          print("Post limit order, sell at price.", pricedata[0])
          enter_trade = False
        except Exception as e:
          print(e)
          enter_trade = True

    '''
    if (BB1[0][0] > current_price > BB[0][0]):
        print("Price might go down, sell to cut lost")
        enter("S")

    if (BB1[2][0] < current_price < BB[2][0]):
        print("Price might go up, buy now")
        enter("B")
    '''
    print("Current price: ", pricedata[0])  
    '''
    print("MA bid(low): ", MAB[0])
    print("MA middle: ", MA[0])
    print("MA ask(high): ", MAA[0])
    '''
    print("BBand lower: ", BB[0][0])
    print("BBand middle: ", BB[1][0])
    print("BBand upper ", BB[2][0])

  
    print("                               Bot working, checking...")

    #call this method again every 5 minutes for new price changes
    s.enter(1800, 1, run, (sc,))
    
s.enter(1, 1, run, (s,))

s.run()